In [1]:
!gcloud config set project etsy-bigquery-adhoc-prod


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Updated property [core/project].


In [2]:
%load_ext google.cloud.bigquery

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("explicit_negatives.csv")

In [5]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Gold Query Set,NaN,NaN,NaN,NaN
1,07/28/2023,Congzhe Su,travi scott rug,1.398464e+09,5677224.0
2,07/28/2023,Congzhe Su,wall art,7.501822e+08,6055794.0
3,07/28/2023,Congzhe Su,preppy,1.072823e+09,6901827.0
4,07/28/2023,Congzhe Su,rug,1.184896e+09,7803638.0


In [6]:
df = df.drop(0)
df.columns = ["date", "reporter", "query", "listing_id", "shop_id"]

In [7]:
df = df[["query", "listing_id"]]
df.listing_id = df.listing_id.astype(int)

In [14]:
df.head()

,query,listing_id
1,travi scott rug,1398463810
2,wall art,750182200
3,preppy,1072822761
4,rug,1184895686
5,wall art,1184895686


In [19]:
df.shape

(4601, 2)

In [9]:
df.groupby("listing_id").count().sort_values("query", ascending=False).head(n=20)

,query
listing_id,
1189955105,1090
1430225274,583
1347295099,460
1023099765,431
1420407748,250
1418810594,137
1503671388,74
1203360316,67
1430837875,61


In [13]:
queries = list(set([
    "lemon eye",
    "sourdough starter",
    "cyberpunk 2077",
    "apollo statue",
    "door harp",
    "goblin slayer armor",
    "sailor jupiter",
    "tin fish plate",
    "dream catcher",
    "moon phase wall art",
    "short curtains",
    "Mindfulness Creative Kits",
    "crystals and herbs",
    "norse jewelry",
    "laufey photocards",
    "boho skirts",
    "maximalist decor",
    "oil lamp",
    "vanity shelf",
    "reading lamp",
    "spice rack",
    "carnelian",
    "paramore tshirt",
    "wall tapestry",
    "safety keychain set",
    "moonstone",
    "butter dish",
    "linen skirt",
    "vampire",
    "candle stick holders",
    "skull necklace",
    "mushroom shelf",
    "furniture",
    "vase",
    "dome chandelier",
    "80s prom",
    "mid century modern furniture",
    "pizza",
]))

listings = [
    1532898330,
    1581863927
]

How I picked queries above: 
- print out 3-days of search training data query listing pairs, get queries if listing is one of the three-witches listings
- limit to queries that are not relevant to witch
- verify on etsy search results to see if I observe the listing
- add examples from search-feedback

In [15]:
full_queries = queries + queries
full_listings = [1532898330] * len(queries) + [1581863927] * len(queries)
three_witches_df = pd.DataFrame({"query": full_queries, "listing_id": full_listings})

In [16]:
three_witches_df.head()

,query,listing_id
0,candle stick holders,1532898330
1,Mindfulness Creative Kits,1532898330
2,oil lamp,1532898330
3,reading lamp,1532898330
4,skull necklace,1532898330


In [18]:
three_witches_df.shape

(76, 2)

In [17]:
negative_df = pd.concat([df, three_witches_df])

In [20]:
negative_df.shape

(4677, 2)

In [21]:
negative_df.to_csv("semantic_relevance_negatives.csv")